# Initial - Exploratory Data Analysis


In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('Data/Gold/main.csv')

ModuleNotFoundError: No module named 'matplotlib'

## Data Dimension

In [ ]:
df.shape

(24959, 56)

## Dataframe contents

In [ ]:
df.head()

,playerID,year,stint,tmID,pos,GP,G,A,Pts,PIM,...,SHG_per_game_score,SHA_per_game_score,GWG_per_game_score,SOG_per_game_score,award_count_per_game_score,+/-_score,stint_score,stint_score_comparison,teammates_same_nationality,tsm_vs_prev_stint
0,abdelju01,2009,1,DET,L,50.0,3.0,3.0,6.0,35.0,...,0.5,0.000000,0.000000,0.345497,0.5,0.000000,2.345497,NaN,3,NaN
1,abdelju01,2010,1,DET,L,74.0,7.0,12.0,19.0,61.0,...,0.5,0.999926,0.999926,0.999996,0.5,1.000000,9.699647,NaN,3,no change
2,abdelju01,2011,1,DET,L,81.0,8.0,14.0,22.0,62.0,...,0.5,0.000000,0.913513,0.000000,0.5,0.576923,7.016692,worse,1,fewer
3,abelcl01,1926,1,NYR,D,44.0,8.0,4.0,12.0,78.0,...,0.5,0.500000,0.500000,0.500000,0.5,0.500000,7.999975,NaN,0,NaN
4,abelcl01,1927,1,NYR,D,23.0,0.0,1.0,1.0,28.0,...,0.5,0.500000,0.500000,0.500000,0.5,0.500000,4.918262,NaN,0,no change


## Check for missing values

In [ ]:
df.isnull().sum()

playerID                         0
year                             0
stint                            0
tmID                             0
pos                              0
GP                               0
G                                0
A                                0
Pts                              0
PIM                              0
+/-                              0
PPG                              0
PPA                              0
SHG                              0
SHA                              0
GWG                              0
SOG                              0
stint_index                      0
career_games                     0
total_GP_prior_to_stint          0
percent_through_career           0
longest_stint                    0
award_count                      0
firstName                        0
lastName                         0
height                           0
weight                           0
birthYear                        0
birthCountry        

## Overview of data types of each column in the dataframe

In [ ]:
df.dtypes


playerID                       object
year                            int64
stint                           int64
tmID                           object
pos                            object
GP                            float64
G                             float64
A                             float64
Pts                           float64
PIM                           float64
+/-                           float64
PPG                           float64
PPA                           float64
SHG                           float64
SHA                           float64
GWG                           float64
SOG                           float64
stint_index                     int64
career_games                  float64
total_GP_prior_to_stint       float64
percent_through_career        float64
longest_stint                   int64
award_count                   float64
firstName                      object
lastName                       object
height                        float64
weight      

In [ ]:
df.select_dtypes(include='number')

,year,stint,GP,G,A,Pts,PIM,+/-,PPG,PPA,...,PPG_per_game_score,PPA_per_game_score,SHG_per_game_score,SHA_per_game_score,GWG_per_game_score,SOG_per_game_score,award_count_per_game_score,+/-_score,stint_score,teammates_same_nationality
0,2009,1,50.0,3.0,3.0,6.0,35.0,-11.0,0.0,0.0,...,0.500000,0.500000,0.5,0.000000,0.000000,0.345497,0.5,0.000000,2.345497,3
1,2010,1,74.0,7.0,12.0,19.0,61.0,15.0,0.0,0.0,...,0.500000,0.500000,0.5,0.999926,0.999926,0.999996,0.5,1.000000,9.699647,3
2,2011,1,81.0,8.0,14.0,22.0,62.0,4.0,0.0,0.0,...,0.500000,0.500000,0.5,0.000000,0.913513,0.000000,0.5,0.576923,7.016692,1
3,1926,1,44.0,8.0,4.0,12.0,78.0,0.0,0.0,0.0,...,0.500000,0.500000,0.5,0.500000,0.500000,0.500000,0.5,0.500000,7.999975,0
4,1927,1,23.0,0.0,1.0,1.0,28.0,0.0,0.0,0.0,...,0.500000,0.500000,0.5,0.500000,0.500000,0.500000,0.5,0.500000,4.918262,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24954,2002,2,66.0,4.0,12.0,16.0,34.0,-7.0,2.0,5.0,...,0.378783,0.858576,0.0,0.999978,0.000000,0.953302,0.5,0.250000,5.631681,0
24955,2003,1,65.0,8.0,13.0,21.0,48.0,4.0,4.0,4.0,...,0.769221,0.697428,0.0,0.000000,0.430757,0.843948,0.5,0.800000,6.973762,0
24956,2005,1,57.0,7.0,11.0,18.0,50.0,-12.0,4.0,4.0,...,0.877182,0.795313,0.0,0.000000,0.491214,0.652307,0.5,0.000000,6.322831,0
24957,2006,1,49.0,1.0,5.0,6.0,30.0,-2.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.5,0.500000,1.550582,0


In [ ]:
df.select_dtypes(include='object')

,playerID,tmID,pos,firstName,lastName,birthCountry,stint_score_comparison,tsm_vs_prev_stint
0,abdelju01,DET,L,Justin,Abdelkader,USA,NaN,NaN
1,abdelju01,DET,L,Justin,Abdelkader,USA,NaN,no change
2,abdelju01,DET,L,Justin,Abdelkader,USA,worse,fewer
3,abelcl01,NYR,D,Clarence,Abel,USA,NaN,NaN
4,abelcl01,NYR,D,Clarence,Abel,USA,NaN,no change
...,...,...,...,...,...,...,...,...
24954,zyuzian01,MIN,D,Andrei,Zyuzin,Russia,better,fewer
24955,zyuzian01,MIN,D,Andrei,Zyuzin,Russia,better,no change
24956,zyuzian01,MIN,D,Andrei,Zyuzin,Russia,worse,no change
24957,zyuzian01,CAL,D,Andrei,Zyuzin,Russia,worse,no change


## Data Distributions

In [ ]:
df.hist(figsize=(10, 10))
plt.show()